<a href="https://colab.research.google.com/github/kchenTTP/sql-intro/blob/main/Intro_to_SQL_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Intro to SQL**

Structured Query Language (SQL) is a domain-specific programming language that allow us to store, retrieve, and manipulate data in structured relational databases.


---

## **Keyword Definition**

**Structured Data**: Data that has a predefined format (schema or data model) and fits neatly into a tabular structure. It is organized and easy to store, process, and analyze using traditional database systems.
  - Databases
  - Spreadsheet

**Database**: A structured collection of data organized and managed in a specific way.

**Table**: A collection of related data organized in rows and columns.

**Relationship**: An association between tables based on common attributes, such as `primary key` and `foreign keys`.

**Primary Key**: A unique identifier for each row in a table, used to ensure data integrity and establish relationships with other tables.

**Foreign Key**: A column that refers to the primary key of another table, establishing a relationship between the two tables.


---

## **Preparing Our Workspace**

Since SQL is a language working with databases. To write SQL, we will typically need a **database management system** and a **SQL client**.


### *Database Management System (DBMS)*

A software system that provides tools and services to manage databases. There are several different database management systems out there and we sometimes refer to them as different SQL flavors or dialects. The most popular SQL dialects are:
  - MySQL
  - PostgreSQL
  - SQLite
  - Microsoft SQL Server

> 💡 We will be working with a database file _(chinook.db)_ using SQLite. Let's download the file for today's class first:


In [1]:
import requests
import os

def gdrive_download(file_id: str, save_path: str) -> None:
  if os.path.exists(save_path):
    os.remove(db_path)

  print(f"Downloading file: '{file_id}'", end="", flush=True)

  url = f"https://drive.google.com/uc?export=download&id={file_id}"

  with requests.Session() as session:
    with session.get(url, stream=True) as res:
      res.raise_for_status()
      with open(save_path, "wb") as f:
        for chunk in res.iter_content(chunk_size=32768):
          if chunk:
            f.write(chunk)
            print(".", end="", flush=True)

  print(f"\nFile saved at '{save_path}'!")

def validate_file(file_path: str):
  print(f"Validating '{file_path}': ", end="", flush=True)
  with open(file_path, "rb") as f:
    header = f.read(15)
    if header != b"SQLite format 3":
      preview = header.decode("utf-8")
      print(preview)
      raise TypeError(f"'{file_path}' is not a valid SQLite database.")

  print(f"SQLite 3.x database")

In [ ]:
db_path = "./chinook.db"
file_id = "1iDtcj6o4nv9cJZMezjRFPP2mDIVxqETv"
url = f"https://drive.google.com/uc?export=download&id={file_id}"

gdrive_download(file_id, db_path)
validate_file(db_path)

### *SQL Client*

A tool or interface that allows you to connect to a DBMS and execute SQL queries. This can be a command-line interface (CLI), a graphical user interface (GUI), a web-based interface, or some code.

#### *Setup Helper Functions*

Since we're using Python to connect to our SQL Database, let's set up some helper functions.

> 📒 **NOTE:** Understanding Python is not required for this class but these 3 functions are required to interact with a SQL database.

- **`show_tables`**: Show all table names in database
- **`show_table_info`**: Show column information of a table
- **`query`**: Query the database

> ❗ Run this cell to set up sqlite helper functions!


In [3]:
# @title
import sqlite3
import pandas as pd
from IPython.display import display, HTML


def query(sql_query: str, db: str = db_path):
  with sqlite3.connect(db) as conn:
    cursor = conn.cursor()
    cursor.execute(sql_query)
    results = cursor.fetchall()

    column_names = [description[0] for description in cursor.description]
    df = pd.DataFrame(results, columns=column_names)

    display(HTML(df.to_html(index=False)))

def show_tables(db: str = db_path):
  sql = "SELECT name as table_name FROM sqlite_master WHERE type='table' AND name not in ('sqlite_sequence', 'sqlite_stat1');"
  query(sql, db)

def show_table_info(table_name: str, db: str = db_path):
  sql = f"PRAGMA table_info({table_name});"
  query(sql, db)

### Explore the Database

In [4]:
# Look at the tables in the database
show_tables()

table_name
albums
artists
customers
employees
genres
invoices
invoice_items
media_types
playlists
playlist_track


In [5]:
# Change the name to any other table to look at the column names and info of that table
show_table_info("customers")

cid,name,type,notnull,dflt_value,pk
0,CustomerId,INTEGER,1,None,1
1,FirstName,NVARCHAR(40),1,None,0
2,LastName,NVARCHAR(20),1,None,0
3,Company,NVARCHAR(80),0,None,0
4,Address,NVARCHAR(70),0,None,0
5,City,NVARCHAR(40),0,None,0
6,State,NVARCHAR(40),0,None,0
7,Country,NVARCHAR(40),0,None,0
8,PostalCode,NVARCHAR(10),0,None,0
9,Phone,NVARCHAR(24),0,None,0


---

## **SQL Keywords**


### `SELECT`

Which columns in a table to select, required keyword when performing queries.

> Note: Use special character _**`*`**_ to select every column.

In [6]:
sql = "select 1.0 / 2;"
query(sql)

1.0 / 2
0.5


### `FROM`

From which table we are selecting the columns.

In [ ]:
sql = "select * from genres;"
query(sql)

GenreId,Name
1,Rock
2,Jazz
3,Metal
4,Alternative & Punk
5,Rock And Roll
6,Blues
7,Latin
8,Reggae
9,Pop
10,Soundtrack


### `AS`

Use `AS` to create aliases for columns.

**Syntax:**
```
SELECT column1 AS alias1, column2 AS alias2
FROM table;
```

In [ ]:
sql = """select InvoiceDate as Date, BillingCountry as Country, Total
from invoices;"""
query(sql)

Date,Country,Total
2009-01-01 00:00:00,Germany,1.98
2009-01-02 00:00:00,Norway,3.96
2009-01-03 00:00:00,Belgium,5.94
2009-01-06 00:00:00,Canada,8.91
2009-01-11 00:00:00,USA,13.86
2009-01-19 00:00:00,Germany,0.99
2009-02-01 00:00:00,Germany,1.98
2009-02-01 00:00:00,France,1.98
2009-02-02 00:00:00,France,3.96
2009-02-03 00:00:00,Ireland,5.94


### `DISTINCT`

Removes duplicate rows from the result, leaving only unique values.

**Syntax:**
```
SELECT DISTINCT column1, column2, ...
FROM table;
```

In [ ]:
sql = """select distinct
BillingCountry, BillingCity
from invoices;"""
query(sql)

BillingCountry,BillingCity
Germany,Stuttgart
Norway,Oslo
Belgium,Brussels
Canada,Edmonton
USA,Boston
Germany,Frankfurt
Germany,Berlin
France,Paris
France,Bordeaux
Ireland,Dublin


### Try it yourself!

**Excercise**:

> _1. Check out columns `CustomerId`, `FirstName`, `Country` from the `customers` table._  
> _2. Give `CustomerId` an alias called `Id`._


In [ ]:
sql = """your code goes here"""
query(sql)

In [ ]:
#@title Answer
sql = """select
  CustomerId as Id,
  FirstName as Name,
  Country
from customers;"""
query(sql)

---

## **Clauses**

Clauses provide additional instructions to specify various conditions, constraints, or operations within an SQL statement to modify the behavior of the query.

### `WHERE`

Filter records based on specified **conditions**. We use comparison operators to define the **condition**:

> ### _Comparison Operators_
> - `=`   Equal to
> - `<`   Less than
> - `>`   Greater than
> - `<>`  Not equal to
> - `<=`  Less than or equal to
> - `>=`  Greater than or equal to

**Syntax:**
```
SELECT column1, column2, ...
FROM table_name
WHERE condition;
```

**Only show invoices from Germany**

In [ ]:
sql = """select
    InvoiceDate,
    BillingCountry,
    BillingCity
from invoices
where BillingCountry = 'Germany';"""
query(sql)

InvoiceDate,BillingCountry,BillingCity
2009-01-01 00:00:00,Germany,Stuttgart
2009-01-19 00:00:00,Germany,Frankfurt
2009-02-01 00:00:00,Germany,Berlin
2009-02-11 00:00:00,Germany,Stuttgart
2009-05-05 00:00:00,Germany,Berlin
2009-05-06 00:00:00,Germany,Berlin
2009-06-15 00:00:00,Germany,Berlin
2009-08-08 00:00:00,Germany,Berlin
2009-10-12 00:00:00,Germany,Stuttgart
2010-02-13 00:00:00,Germany,Berlin


### `AND`/ `OR`

Use `AND` and `OR` to add more conditions to your queries.

**Syntax:**
```
condition1 AND condition2
```

**Show invoices from Germany but not from Berlin**

In [ ]:
sql = """select
    InvoiceDate,
    BillingCountry,
    BillingCity
from invoices
where BillingCountry = 'Germany' OR BillingCity <> 'Berlin';
"""
query(sql)

InvoiceDate,BillingCountry,BillingCity
2009-01-01 00:00:00,Germany,Stuttgart
2009-01-02 00:00:00,Norway,Oslo
2009-01-03 00:00:00,Belgium,Brussels
2009-01-06 00:00:00,Canada,Edmonton
2009-01-11 00:00:00,USA,Boston
2009-01-19 00:00:00,Germany,Frankfurt
2009-02-01 00:00:00,Germany,Berlin
2009-02-01 00:00:00,France,Paris
2009-02-02 00:00:00,France,Bordeaux
2009-02-03 00:00:00,Ireland,Dublin


### `BETWEEN`
Used to specify a range of values for comparison.

**Syntax:**
```
column_name BETWEEN value1 AND value2
```

**Show album sales between 2010 and 2012**

In [ ]:
sql = """select date(InvoiceDate) as Date, CustomerId, Total
from invoices
where date(InvoiceDate) between '2010' AND '2012';"""
query(sql)

Date,CustomerId,Total
2010-01-08,43,1.98
2010-01-08,45,1.98
2010-01-09,47,3.96
2010-01-10,51,6.94
2010-01-13,57,17.91
2010-01-18,7,18.86
2010-01-26,21,0.99
2010-02-08,22,1.98
2010-02-08,24,1.98
2010-02-09,26,3.96


### `IN`

Allows you to match a column's value against a list of specified values.

**Syntax:**
```
column_name IN (value1, value2, ...)
```

**Find employees who are in manager positions**

In [ ]:
sql = """select
    FirstName || ' ' || LastName as name,
    Title
from employees
where Title in ('General Manager', 'Sales Manager', 'IT Manager');"""
# where Title LIKE '% Manager';
query(sql)

name,Title
Andrew Adams,General Manager
Nancy Edwards,Sales Manager
Michael Mitchell,IT Manager


### _Functions_

Functions are pre-defined operations that perform specific computations or transformations on data.

#### [**`date(timestring, [modifiters], [...])`**](https://www.sqlitetutorial.net/sqlite-date-functions/sqlite-date-function/)
The `date()` function accepts a time string and zero or more modifiers as arguments. It returns a date string in this format: **`YYYY-MM-DD`**.

#### [**`string1 || string2`**](https://www.sqlitetutorial.net/sqlite-string-functions/sqlite-concat/)
Concatenate two strings into a single string. Can be chained together to concatenate more than 2 strings.

### Try it yourself!

**Excercise**:

**_Find invoices from USA or Canada where the total amount spent is greater than $8_**
> _Show  **`InvoiceId`**, **`InvoiceDate`**, **`BillingCountry`**, **`Total`**  columns from the **`invoices`** table._


In [ ]:
sql = """your code goes here"""
query(sql)

In [ ]:
#@title Answer
sql = """select InvoiceId, InvoiceDate, BillingCountry, Total
from invoices
where BillingCountry in ('USA', 'Canada') and Total > 8;"""
query(sql)

---
### `GROUP BY`

Group rows based on one or more columns into subgroups. Typically used in conjunction with **_aggregate functions_** to perform calculations or summary operations on groups of rows.

**Syntax:**
```
SELECT column1, column2, ..., aggregate_function(column)
FROM table_name
GROUP BY column1, column2, ...;
```

### _Aggregate Functions_
Functions that operate on a set of rows and calculate a single value as the result.

#### `COUNT(column)`

Counts the number of rows in a column.

**Count the number of employees in different titles**

In [7]:
sql = """select
    Title,
    count(*) as Number
from employees
group by Title;"""
query(sql)

Title,Number
General Manager,1
IT Manager,1
IT Staff,2
Sales Manager,1
Sales Support Agent,3


#### `SUM(column)`

Sums up the numerical values in a column.

**Show yearly sale numbers**

In [ ]:
sql = """select
    strftime('%Y', InvoiceDate) as year,
    sum(Total) as total
from invoices
group by year;"""
query(sql)

year,total
2009,449.46
2010,481.45
2011,469.58
2012,477.53
2013,450.58


### `ORDER BY`

Change the order of appearance.

**Syntax:**
```
ORDER BY column sorting_order
```

> - **sorting_order**:
>   - **ASC**: Ascending order
>   - **DESC**: Descending order

### `LIMIT`

Limit the amount of output to the specified number.

**Syntax:**
```
LIMIT number
```

In [ ]:
sql = """select Name
from artists
order by name desc
limit 10;"""
query(sql)

Name
Zeca Pagodinho
Youssou N'Dour
Yo-Yo Ma
Yehudi Menuhin
Xis
Wilhelm Kempff
Whitesnake
Vinícius E Qurteto Em Cy
Vinícius E Odette Lara
Vinícius De Moraes & Baden Powell


### Try it yourself!

**Excercise 1**:

**_Find top 10 artists that have the most amount of tracks_**
> _1. Show the `Composer` and the number of tracks they have._  
> _2. Remove `NULL` values._
>
> _Hint: Use the `tracks` table._

In [ ]:
sql = """your code goes here"""
query(sql)

In [ ]:
#@title Answer
sql = """select Composer, count(*) as n_tracks
from tracks
where Composer is not null
group by Composer
order by n_tracks desc
limit 10;"""
query(sql)

**Excercise 2**:

**_Find artists that have the top 10 longest track_**
> _1. Show the `Composer` and the length of their longest track tracks (`Milliseconds`)._  
> _2. Remove `NULL` values._
>
> _Hint: Use the `tracks` table._

In [ ]:
sql = """your code goes here"""
query(sql)

In [ ]:
#@title Answer
sql = """select Composer, max(Milliseconds) as Milliseconds
from tracks
where Composer is not null
group by Composer
order by Milliseconds desc
limit 10;"""
query(sql)

---

## **Joins**

Joins are used to combine rows from two tables based on related columns that has **matching values**. Commonly, joins are performed when there is a relationship between tables, such as a **primary key** in one table that corresponds to a **foreign key** in another table.

### _Join Condition_

By specifying the columns used to link the tables, or the _**join condition**_, you can create a result set that includes data from multiple tables as per the defined relationship.

<br>

<figure align="center">
  <img src="https://raw.githubusercontent.com/kchenTTP/sql-intro/refs/heads/main/assets/sql-joins.jpg" alt="SQL Joins" />
  <figcaption>The 4 kinds of SQL Joins<figcaption>
</figure>



### `INNER JOIN`

Returns only the rows that have matching values in both tables.

**Syntax:**
```
SELECT t1.column1, t2.column2
FROM table1 t1
INNER JOIN table2 t2 ON t1.id = t2.id;
```

**Show customers that have spent the most amount of money**

In [ ]:
sql = """select
    c.FirstName || ' ' || c.LastName as name,
    sum(i.Total) as total_spent,
    count(i.InvoiceDate) as n_times_shopped
from customers c
inner join invoices i on c.CustomerId = i.CustomerId
group by name
order by total_spent desc;"""
query(sql)

name,total_spent,n_times_shopped
Helena Holý,49.62,7
Richard Cunningham,47.62,7
Luis Rojas,46.62,7
Ladislav Kovács,45.62,7
Hugh O'Reilly,45.62,7
Julia Barnett,43.62,7
Fynn Zimmermann,43.62,7
Frank Ralston,43.62,7
Victor Stevens,42.62,7
Astrid Gruber,42.62,7


### Try it yourself!

**Excercise**:

**_Find top 20 artists that sold the most amount of tracks_**

> _Hint:_  
> *1. Both `invoice_items` and `tracks` table have a column named `TrackId`.*  
> _2. Use `GROUP BY` to group rows into subgroups of `Composers`._  
> *3. Each row in `invoice_items` is one sale.*  
> _4. Remove `NULL` values._


In [ ]:
sql = """your code goes here"""
query(sql)

In [ ]:
#@title Answer
sql = """select
t.Composer as artist,
count(ii.InvoiceId) as n_tracks_bought
from invoice_items ii
join tracks t on t.TrackId = ii.TrackId
where t.composer is not null
group by artist
order by n_tracks_bought desc
limit 20;"""
query(sql)

### `LEFT JOIN`

Returns the rows that have matching values in both tables and every row on the left table.

**Syntax:**
```
SELECT t1.column1, t2.column2
FROM table1 t1
LEFT JOIN table2 t2 ON t1.id = t2.id;
```

### `RIGHT JOIN`

Returns the rows that have matching values in both tables and every row on the right table.

**Syntax:**
```
SELECT t1.column1, t2.column2
FROM table1 t1
RIGHT JOIN table2 t2 ON t1.id = t2.id;
```

**Show the least popular artists in the record store**

> _💡 Without using `LEFT JOIN`, we won't be able to see artists with 0 sales. Since only the ones who have sold tracks will have a record in the `invoiceitems` table_

In [9]:
sql = """select
    t.Composer as artist,
    count(ii.InvoiceId) as n_tracks_bought
from tracks t
left join invoice_items ii on t.TrackId = ii.TrackId
where t.composer is not null
group by artist
order by n_tracks_bought asc;"""
query(sql)

artist,n_tracks_bought
A. Jamal,0
Aaron Copland,0
Aaron Goldberg,0
Acyr Marques/Arlindo Cruz/Franco,0
Adrian Smith/Bruce Dickinson/Nicko McBrain,0
Adrian Vanderberg/David Coverdale,0
Adriano Bernandes/Edmar Neves,0
Alba Carvalho/Fernando Porto,0
Albert King,0
Alcides Dias Lopes,0


### Try it yourself!

**Excercise**:

**_Find the most popular genre and least popular genre in the store_**

> _Hint:_  
> _1.Popularity is based on how many times a track of a genre is bought._  
> *2. `invoice_items` table contains data on the tracks associated with invoices.*  
> _3. `tracks` table has a `GenreId` foreign key from `genres` table, which contains the genre names._  
> _4. Use `GROUP BY` to group rows into subgroups of `Genres`._  
> _5. Be aware of genres that does not have any sales._


In [ ]:
sql = """your code goes here"""
query(sql)

In [ ]:
#@title Answer
sql = """select distinct
  g.Name as genre,
  count(ii.InvoiceId) as n_tracks_bought
from genres g
join tracks t on t.GenreId = g.GenreId
left join invoice_items as ii on t.TrackId = ii.TrackId
group by genre
order by n_tracks_bought desc;"""
query(sql)

### `FULL OUTER JOIN`

Returns the rows that have matching values in both tables and every single row in both tables.

**Syntax:**
```
SELECT t1.column1, t2.column2
FROM table1 t1
FULL OUTER JOIN table2 t2 ON t1.id = t2.id;
```


> 🚨 Unfortunately, full outer join is not supported by the Python `sqlite3` library yet.

**Show every artist and track, and match track to artist if there is a match**

In [ ]:
# sql = """select
#   a.Name as Artist,
#   t.Composer,
#   t.Name as Track
# from artists a
# full outer join tracks t
# on a.Name = t.Composer;"""
# query(sql)

---

## **Resources**

For more information on Sqlite, please visit [SQLite Tutorial](https://www.sqlitetutorial.net/) or [SQLite Documentation](https://www.sqlite.org/docs.html)


---

## **Class Survey**

Directly below is a link to a survey that lets us know how we're doing and any additional feedback you might have. Please take some time to fill out the survey in full so that we can learn how to better serve you.

[**Survey Link**](https://docs.google.com/forms/d/e/1FAIpQLSeRoFsj9kC436jyBuImwv2QToGSYYZDo1SygTEnsQ-k3ozHng/viewform)